In [ ]:
#@title ←Run <Center>Mount Google Drive </Center>
#@markdown <Center> <img width="80" src="https://kstatic.googleusercontent.com/files/e20dbc1db7b6eab22fb2d07df8043f8a74b301dbb8c176e6a1992b2dc229c5ae94b5c260cf549c41712fa40b7e639166c6a8461303a27c2a3b9c55b8ca155ec3" class="icon" alt="Icon">

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ←Run <Center>Initialize Send.cm Downloader </Center>
#@markdown <Center> <img width="80" src="https://send.cm/static/img/logo.png" class="icon" alt="Icon">

# Installing Packages
!pip install requests
!pip install bs4
!pip install clint
!pip install lxml

# Imports 
import requests
from clint.textui import progress
import sys
import os
from bs4 import BeautifulSoup

# initalizing the Shell
if not os.path.exists("/root/.ipython/ocr.py"): 
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh
runSh('apt install -y aria2')

# Code
class FileDownloader:
    """
    Download a File from a link
    """

    def __init__(self, dest_folder, noprogress=False) -> None:
        self.session = requests.Session()
        self.dest_folder = dest_folder
        self.noprogress= noprogress


    def download_request(self, url, form_encoded):
        if form_encoded != None:
            redirected_response = self.session.post(url, data=form_encoded,
                headers={'Content-Type': 'application/x-www-form-urlencoded'},
                allow_redirects=False)
            return redirected_response.headers["Location"]
        return url

    def download(self, url:str, name:str="file", form_encoded=None):
        """
        Downloads a file
        """

        url = self.download_request(url, form_encoded=form_encoded)
        filepath=f"{self.dest_folder}/{name}"
        runSh(f'aria2c -x 16 -s 16 -k 1M -d {self.dest_folder} {url}', output=True)
  


class SendcmDownloader:
    """Downloads Send.cm Folders
    """
    def __init__(self, folder_link:str, dest_folder:str, noprogress=False) -> None:
        """ 
        Args:
            folder_link (str): The send.cm folder link
            dest_folder (str): The local destination folder
        """
        self.session = requests.Session()
        self.folder_link = folder_link
        folder_name = folder_link.rsplit("/",1)[-1]
        self.dest_folder = f"{dest_folder}/{folder_name}"
        if not os.path.exists(self.dest_folder):
            os.mkdir(self.dest_folder)
        self.file_downloader = FileDownloader(self.dest_folder, noprogress=noprogress)

    def get_folder_content(self):
        """Gets all the files in a Folder
        """
        url = self.folder_link
        base="https://send.cm"
        done = False
        while not done:
            response = self.session.get(url)
            if response.status_code != 200:
                print("Error Loading Page!")
                sys.exit(0)
            soup  = BeautifulSoup(response.content, "lxml")

            table = soup.find("table", id="xfiles")
            files = table.find_all("a", class_="tx-dark") # these are files
            for file in files:
                self.download_file(file["href"], file.string)
            pagination = soup.find("ul",class_="pagination")
            if pagination == None:
                done = True
            else :
                current_page = pagination.find("li", "page-item actived", recursive=False)
                next_page = current_page.next_sibling # if there is pagination
                if next_page == None:
                    done = True
                else:
                    url = base + next_page["href"]
            
    def download_file(self, url:str, file_name:str):
        """Downloads a file

        Args:
            url (str): file url
            file_name (str): the file name & its extension
        """
        print(f"{file_name}:{url}")
        
        form_encoded = {
            "op":"download2",
            "id": url.rsplit("/")[-1],
            "referer": self.folder_link
            }
        download_link = "https://send.cm/"

        self.file_downloader.download(download_link, file_name, form_encoded)




if __name__ == "__main__":
    args = sys.argv[1:] 
    dest_folder = ""
    noprogress=False
    if len(args) == 0:
        print("Folder Link is required!")
        sys.exit(0)

    folder_link = args[0]    
    if (len(args) >= 2):
        if args[1] == '--noprogress':
            noprogress = True
        else :
            dest_folder = args[1]

    if (len(args) >=3):
        if args[2] == '--noprogress':
            noprogress = True 


    

In [ ]:
#@title ←Run Downloader

folder_link = "https://send.cm/s/17R/Stripe_Payments_JavaScript_Course" #@param {type:"string"}
dest_folder = "/content/drive/MyDrive/" #@param {type:"string"}

downloader = SendcmDownloader(folder_link, dest_folder)
downloader.get_folder_content()